# Sudoku solver with statistics

Author: Frankie Inguanez <br />
Date: 15/01/2023<br /><br />

An backtracking algorithm for a 9x9 sudoku puzzle taking.

In [1]:
def to2DArray(n):
    """
    Convert a string to a 2D 9x9 array.
    Arguments:
        n: an 81 character string to be converted to a 2D array representing a 9x9 sudoku grid.
    """
    return [list(map(int, n[i:i+9])) for i in range(0, 81, 9)]

def toStr(puzzle):
    """
    Converts a puzzle to a string
    Arguments:
        puzzle: A 2D array with a 9x9 sudoku grid.
    """
    r = ""

    for row in puzzle:
        r += "".join(map(str, row))

    return r

def getColValues(puzzle, col):
    """
    Get column values
    Arguments:
        puzzle: A 2D array with a 9x9 sudoku grid.
        col: the column number
    """
    lst = []
    for row in puzzle:
        lst.append(row[col])

    return lst

def getBoxValues(puzzle, box):
    """
    Get box values. Boxes are 3x3 sub-grids enumerates from top left in a raster fashion
    0, 1, 2
    3, 4, 5
    6, 7, 8
    Arguments:
        puzzle: A 2D array with a 9x9 sudoku grid.
        box: the box number
    """
    
    return [puzzle[x][y] for x in range((box//3)*3,((box//3)*3)+3) for y in range((box%3)*3, ((box%3)*3)+3)]

def checkList(lst):
    """
    Check if a list of digits contain all values from 1 to 9
    Arguments:
        lst: the list to check.
    """
    return set(lst) == set(range(1,10))

def isSolved(puzzle):
    """
    Checks if a 9x9 sudoku puzzle has been solved.
    Arguments:
        puzzle: A 2D array with a 9x9 sudoku grid.
    """
    # Check rows
    for row in puzzle:
        if not checkList(row):
            return False

    # Check columns
    for i in range(0,9):
        if not checkList(getColValues(puzzle, i)):
            return False;

    # Check box
    for i in range(0,9):
        if not checkList(getBoxValues(puzzle, i)):
            return False;

    return True

def isValid(puzzle, num, pos):
    """
    Checks if a number can be added to a specific position
    Arguments:
        puzzle: A 2D array with a 9x9 sudoku grid.
        num: the number to be inserted
        pos: the row and column position to be considered.
    """

    # Check the row
    for i in range(len(puzzle[0])):
        if puzzle[pos[0]][i]==num and pos[1] != i:
            return False

    # Check the column
    for i in range(len(puzzle[1])):
        if puzzle[i][pos[1]]==num and pos[0] != i:
            return False

    # Check box
    box_x = pos[1] // 3
    box_y = pos[0] // 3

    for i in range(box_y*3, box_y*3 + 3):
        for j in range(box_x * 3, box_x*3 + 3):
            if puzzle[i][j] == num and (i,j) != pos:
                return False

    return True

def getFileLineCount(fileName: str):
    """
    Get number of lines in a file.
    Arguments:
        fileName: the name of the file to process.
    """
    import mmap

    lines = 0
    with open(fileName, "r+", encoding="utf-8") as f:
        bf = mmap.mmap(f.fileno(), 0)

        while bf.readline():
            lines += 1

    return lines

def saveError(error, errorsFileName: str):
    """
    Saves an error/exception that is raised.
    Arguments:
        error: the Exception that is raised.
        errorsFileName: the file name where the error will be saved.
    """
    try:
        with open(errorsFileName, "a", encoding="utf-8") as ef:
            ef.write("Encountered error:\n{}\n{}\n{}\n\n".format(type(error), error.args, error))
    except Exception as e:
        # Failed to save error to file
        print("Failed to save original error to file due to:\n{}\n{}\n{}\n\n".format(type(e), e.args, e))
        print("Original error:\n{}\n{}\n{}\n\n".format(type(error), error.args, error))

In [2]:
def findRandom(puzzle):
    """
    Finds the next empty cell in a random fashion.
    """
    import random

    for row in random.sample(range(0,9),9):
        for col in random.sample(range(0,9),9):
            if puzzle[row][col] == 0:
                return (row, col)

    return None

def findByRow(puzzle):
    """
    Finds the next empty cell in a raster fashion, row by row.
    Arguments:
        puzzle: a 9x9 sudoku puzzle
    """
    
    for row in range(len(puzzle)):
        for col in range(len(puzzle[0])):
            if puzzle[row][col] == 0:
                return (row, col)
            
    return None

def findByCol(puzzle):
    """
    Finds the next empty cell in a column order.
    Arguments:
        puzzle: a 9x9 sudoku puzzle
    """
    for col in range(0,9):
        for row in range(len(puzzle)):
            if puzzle[row][col]==0:
                return (row, col)
    
    return None

def findByBox(puzzle, mode):
    """
    Finds the next empty cell searching first by box then by row.
    Arguments:
        puzzle: a 9x9 sudoku puzzle
        mode:   4 searches for boxes sequentially, 
                5 searches for boxes in a zig-zag fashion, 
                6 searches for boxes in spiral fashion, 
                7 searches for boxes in a semi zig-zag fashion,
                8 searches for boxes randomly
    """
    import random

    if mode==4:
        boxes=range(0,9)
    elif mode==5:
        boxes=[0,1,2,5,4,3,6,7,8]
    elif mode==6:
        boxes=[0,1,2,5,8,7,6,3,4]
    elif mode==7:
        boxes=[0,1,4,3,6,7,8,5,2]
    elif mode==8:
        boxes=random.sample(range(0,9),9)
    elif mode==9:
        boxes=[0,4,8,1,2,3,5,6,7]

    for box in boxes:
        for row in range((box//3)*3,((box//3)*3)+3):
            for col in range((box%3)*3, ((box%3)*3)+3):
                if puzzle[row][col]==0:
                    return (row, col)

    return None

def findEmpty(puzzle, searchMode):
    """
    Finds the next empty cell in a 9x9 sudoku puzzle.
    Arguments:
        puzzle: the 9x9 sudoku puzzle.
        searchMode: defines how the puzzle is parsed: 
                    1 by row; 
                    2 by col; 
                    3 random;
                    4 by box sequentially; 
                    5 by box in a zig-zag; 
                    6 by box in a spiral; 
                    7 by box in a semi-zig-zag;
                    8 by box randomly;
                    9 by box diagonal;
    """
    if searchMode==1:
        return findByRow(puzzle)
    elif searchMode==2:
        return findByCol(puzzle)
    elif searchMode==3:
        return findRandom(puzzle)
    elif searchMode>=4 and searchMode<=9:
        return findByBox(puzzle, searchMode)
    
    return None

In [3]:
def getGuesses(puzzle, guessMode):
    """
    Gets numbers to guess.
    Arguments:
        puzzle: the 9x9 sudoku puzzle.
        guessMode: the guessing mode. 1 for sequential, 2 for random.
    """
    import random

    if guessMode==1:
        return range(1,10)
    elif guessMode==2:
        return random.sample(range(1,10),9)

    return None

In [4]:
class SudokuExecutor:
    def __init__(self, puzzlesFileName: str, statsFileName: str, errorsFileName: str, offset: int, limit: int):
        self.puzzlesFileName=puzzlesFileName
        self.statsFileName=statsFileName
        self.errorsFileName=errorsFileName
        self.offset=offset
        self.limit=limit

In [5]:
class SudokuConfig:
    def __init__(self, searchMode: int, guessMode: int):
        self.searchMode=searchMode
        self.guessMode=guessMode

In [6]:
class SudokuStats:
    def __init__(self):
        self.guesses = 0
        self.backtracks = 0
        self.executionTime = None
        self.unknowns = None

    def incrementGuesses(self):
        self.guesses += 1

    def incrementBacktracks(self):
        self.backtracks += 1

    def registerExecutionTime(self, executionTime):
        self.executionTime=executionTime

    def setUnknowns(self, zeros:int):
        self.unknowns=zeros

In [7]:
def backtracking(board, stats: SudokuStats, config: SudokuConfig):
    """
    Solves a 9x9 sudoku puzzle using backtracking algorithm.
    Arguments:
        board: the 9x9 puzzle to be solved.
        stats: The statistics object to record algorithm.
        searchMode: defines how the puzzle is parsed: 1 by row; 2 by col; 3 by box sequentially; 4 by box in a zig-zag; 5 by box in a spiral; 6 by box in a semi-zig-zag
        guessMode: defines how numbers are guessed: 1 sequentially; 2 randomly
    """
    # Find the next empty cell
    find = findEmpty(board, config.searchMode)

    # If there is no empty cell than puzzle is complete
    if not find:
        return True
    else:
        row, col = find

    # Get numbers to guess and attempt
    for guess in getGuesses(board, config.guessMode):
        if isValid(board, guess, (row, col)):

            # Brute force guess
            stats.incrementGuesses()
            board[row][col] = guess

            # Attempt to solve rest of puzzle with current choice
            if backtracking(board, stats, config):
                return True

            # Invalid puzzle so backtrack
            stats.incrementBacktracks()
            board[row][col] = 0

    return False

In [8]:
board = to2DArray('100008509000000000460000000600000000085020047094600302000003090920000700000107005')
board

[[1, 0, 0, 0, 0, 8, 5, 0, 9],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [4, 6, 0, 0, 0, 0, 0, 0, 0],
 [6, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 8, 5, 0, 2, 0, 0, 4, 7],
 [0, 9, 4, 6, 0, 0, 3, 0, 2],
 [0, 0, 0, 0, 0, 3, 0, 9, 0],
 [9, 2, 0, 0, 0, 0, 7, 0, 0],
 [0, 0, 0, 1, 0, 7, 0, 0, 5]]

In [9]:
# Register the start time
config = SudokuConfig(searchMode=6, guessMode=1)

print("Solved puzzle!" if backtracking(board, SudokuStats(), config) else "Could not solve puzzle")

Solved puzzle!


In [10]:
import timeit

stats = SudokuStats()

timeit.timeit(lambda: backtracking(board, stats, config), number=100000)

1.3035030999890296

In [11]:
stats = SudokuStats()
stats.registerExecutionTime(timeit.timeit(lambda: backtracking(board, stats, config), number=100000))

In [12]:
def solvePuzzles(executor: SudokuExecutor, config: SudokuConfig):
    """
    Solves puzzles found in a file using backtracking algorithm.
    Arguments:
        puzzlesFileName: file containing puzzles in following format: <id>, <puzzle>, <solution>
        statsFileName: file where statistics shall be saved.
        errorsFileName: file where errors shall be saved.
        limit: the limit number of puzzles to solve.
        searchMode: defines how missing values are searched.
        guessMode: defines how guesses are made.
    """
    import tqdm
    import timeit

    if not executor.limit:
        limit = getFileLineCount(executor.puzzlesFileName)
    else: limit = executor.limit

    i = 1
    hasError = False
    try:
        print("Starting sudoku base solver.")
        
        # Open statistics file
        with open(executor.statsFileName, "w", encoding="utf-8") as sf:
            #Write header row
            sf.write("Puzzle,Solution,Execution Time,Zeros,Guesses,Backtracks\n")

            # Open puzzles and read till limit is reached.
            with open(executor.puzzlesFileName, "r", encoding="utf-8") as pf:
                for line in tqdm.tqdm(pf, total=limit):
                    if i < executor.offset:
                        i += 1
                        continue

                    # Stop at limit
                    if (i>limit):
                        break
                    
                    # Parse the content
                    p = line.strip()

                    # Create board and statistics
                    board = to2DArray(p)
                    
                    stats = SudokuStats()
                    stats.setUnknowns(p.count('0'))
                    stats.registerExecutionTime(timeit.timeit(lambda: backtracking(board, stats, config), number=1000))                   
                        
                    # Write statistics
                    sf.write("{},{},{:0.17f},{:0.0f},{:0.0f},{:0.0f}\n".format(p, toStr(board), stats.executionTime, stats.unknowns, stats.guesses, stats.backtracks))
                    i+=1

    except Exception as e:
        hasError = True
        saveError(e, executor.errorsFileName)

    print("Operation encountered some errors. Check {} for details or script output above.".format(executor.errorsFileName) \
        if hasError else "Sudoku puzzles solved completed successfully.")
    print("Sudoku solver statistics saved in {}".format(executor.statsFileName))

In [13]:
config = SudokuConfig(searchMode=1, guessMode=1)

exec = SudokuExecutor(puzzlesFileName="puzzles.txt",statsFileName="backtracking.csv", \
                    errorsFileName="backtrackingErrors.txt", offset=0, limit=5)
solvePuzzles(exec, config)

Starting sudoku base solver.


100%|██████████| 5/5 [00:00<00:00, 119.03it/s]

Sudoku puzzles solved completed successfully.
Sudoku solver statistics saved in backtracking.csv
